In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRFRegressor


In [3]:
df = pd.read_csv(r"D:\mlproject\notebook\data\stud.csv")

In [4]:
X = df.drop(columns=['math_score'],axis=1)
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [5]:
print("Categories in 'gender' variable:",end=" " )
print(df['gender'].unique())

Categories in 'gender' variable: ['female' 'male']


In [6]:
y = df['math_score']
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

In [7]:
num_feature = X.select_dtypes(exclude="object").columns
cat_feature = X.select_dtypes(include="object").columns


In [8]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer


In [9]:
num_transformer = StandardScaler()
on_transformer = OneHotEncoder()
preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder",on_transformer,cat_feature),
        ("StandardScaler",num_transformer,num_feature)
    ]
)

In [10]:
X = preprocessor.fit_transform(X)
X.shape

(1000, 19)

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape,X_test.shape

((800, 19), (200, 19))

In [20]:
def evaluate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mse)
    r2_scor = r2_score(true,predicted)
    return mae,rmse,r2_scor


In [22]:
models = {
    "Linear Regression":LinearRegression(),
    "Lasso": Lasso(),
    "Ridge":Ridge(),
    "K-Neighbour Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor":XGBRFRegressor(),
    "CatBoosting Regressor":CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae,model_train_rmse, model_train_r2 = evaluate_model(y_train,y_train_pred)

    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    

    print("model performance for trainig set:")
    print("mean absolute error is:", model_train_mae)
    print("root mean squared error is:",model_train_rmse)
    print("r2 score is:",model_train_r2)

    print('----------------------------------')

    print("model performancr for test set is:")
    print("mean absolute error is:", model_test_mae)
    print("root mean squared error is:",model_test_rmse)
    print("r2 score is:",model_test_r2)
    r2_list.append(model_test_r2)

    print('='*35)
    print('\n')




Linear Regression
model performance for trainig set:
mean absolute error is: 4.267138671875
root mean squared error is: 5.324354270503339
r2 score is: 0.8742556463650867
----------------------------------
model performancr for test set is:
mean absolute error is: 4.2158203125
root mean squared error is: 5.3959872842671395
r2 score is: 0.8803449074540941


Lasso
model performance for trainig set:
mean absolute error is: 5.206302661246526
root mean squared error is: 6.593815587795566
r2 score is: 0.8071462015863456
----------------------------------
model performancr for test set is:
mean absolute error is: 5.157881810347763
root mean squared error is: 6.519694535667419
r2 score is: 0.8253197323627853


Ridge
model performance for trainig set:
mean absolute error is: 4.26498782372598
root mean squared error is: 5.323324922741654
r2 score is: 0.8743042615212909
----------------------------------
model performancr for test set is:
mean absolute error is: 4.211100688014261
root mean squared

In [23]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=['model','r2_score']).sort_values(by=['r2_score'],ascending=False)

,model,r2_score
2,Ridge,0.880593
0,Linear Regression,0.880345
6,XGBRegressor,0.862432
5,Random Forest Regressor,0.852559
7,CatBoosting Regressor,0.851632
8,AdaBoost Regressor,0.851013
1,Lasso,0.825320
3,K-Neighbour Regressor,0.783813
4,Decision Tree,0.753615
